In [94]:
from fastembed import TextEmbedding
import numpy as np
import requests
from qdrant_client import QdrantClient, models
import uuid 

### Q1. Embedding the query

In [27]:
query = 'I just discovered the course. Can I join now?'

In [28]:
model_name = 'jinaai/jina-embeddings-v2-small-en'

In [29]:
embedding_model = TextEmbedding(model_name=model_name)

In [30]:
query_embedding_list = list(embedding_model.embed([query]))

In [ ]:
if query_embedding_list:
    query_embedding = query_embedding_list[0]
    print(f"Embedding shape: {query_embedding.shape}")

    min_value = np.min(query_embedding)
    print(f"Minimal value in the embedding: {min_value}")

    max_value = np.max(query_embedding)
    norm_value = np.linalg.norm(query_embedding)
    print(f"Maximal value in the embedding: {max_value}")
    print(f"Norm of the embedding: {norm_value}")
else:
    print("Failed to generate embedding.")

Embedding shape: (512,)
Minimal value in the embedding: -0.11726373885183883
Maximal value in the embedding: 0.13307955818198478
Norm of the embedding: 1.0


### Q2. Cosine similarity with another vector

In [24]:
doc = "Can I still join the course after the start date"

In [32]:
doc_embedding_list = list(embedding_model.embed([doc]))

In [33]:
if doc_embedding_list:
    doc_embedding = doc_embedding_list[0]

    cosine_similarity = np.dot(query_embedding, doc_embedding)
    print(f"Cosine similarity for Q2: {cosine_similarity}")
else:
    print("Failed to generate document embedding for Q2.")

Cosine similarity for Q2: 0.9003211830791032


### Q3. Ranking by cosine

In [34]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [35]:
# Extract text fields for embedding
doc_texts = [doc['text'] for doc in documents]

In [37]:
# Embed all documents
doc_embeddings = list(embedding_model.embed(doc_texts))

In [39]:
if not all(isinstance(emb, np.ndarray) for emb in doc_embeddings):
    print("Failed to generate some document embeddings for Q3.")
    exit()

In [40]:
similarities = [np.dot(query_embedding, doc_emb) for doc_emb in doc_embeddings]


highest_similarity_index = np.argmax(similarities)

print(f"Similarities for Q3: {similarities}")
print(f"Index of document with highest similarity for Q3: {highest_similarity_index}")

Similarities for Q3: [np.float64(0.762968451872193), np.float64(0.8182378156620136), np.float64(0.8085397445747489), np.float64(0.7133078832064158), np.float64(0.7304499196411823)]
Index of document with highest similarity for Q3: 1


### Q4. Ranking by cosine, version two

In [43]:
full_text = [doc['question'] + ' ' + doc['text'] for doc in documents]

In [44]:
# Embed all documents
full_text_embeddings = list(embedding_model.embed(full_text))

In [45]:
if not all(isinstance(emb, np.ndarray) for emb in full_text_embeddings):
    print("Failed to generate some document embeddings for Q4.")
    exit()

In [46]:
similarities = [np.dot(query_embedding, doc_emb) for doc_emb in full_text_embeddings]


highest_similarity_index = np.argmax(similarities)

print(f"Similarities for Q4: {similarities}")
print(f"Index of document with highest similarity for Q4: {highest_similarity_index}")

Similarities for Q4: [np.float64(0.851454319443226), np.float64(0.8436594005975433), np.float64(0.8408287224005012), np.float64(0.7755157657626952), np.float64(0.8086007917931166)]
Index of document with highest similarity for Q4: 0


### Q5. Selecting the embedding model

In [49]:
models = TextEmbedding.list_supported_models()

In [53]:
models


[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [54]:
# Find the model with the smallest 'dim'
min_model = min(models, key=lambda x: x.get("dim", float("inf")))

# Display result
print(f"Lowest dimension model: {min_model['model']} with {min_model['dim']} dimensions")

Lowest dimension model: BAAI/bge-small-en with 384 dimensions


### Q6. Indexing with qdrant 

In [56]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [57]:
qd_client = QdrantClient("http://localhost:6333")

In [58]:
model_handle = "BAAI/bge-small-en"

In [77]:
embedding_model = TextEmbedding(model_name=model_handle)

In [61]:
EMBEDDING_DIMENSIONALITY = 384

In [59]:
collection_name = "zoomcamp-hw"

In [62]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [63]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [64]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [65]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [66]:
question = 'I just discovered the course. Can I join now?'

In [78]:
question_embedding_list = list(embedding_model.embed([question]))

In [81]:
query_vector = question_embedding_list[0].tolist()

In [80]:
print(query_vector.shape)

(384,)


In [93]:
try:
    search_results = qd_client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=5 
    )

    if search_results:
        highest_score = search_results[0].score
        print(f"\nSearch results for question: '{question}'")
        print(f"\nHighest score in the results for Q6: {highest_score}")
    else:
        print("No results found for the question.")

except Exception as e:
    print(f"An error occurred during search: {e}")


Search results for question: 'I just discovered the course. Can I join now?'

Highest score in the results for Q6: 0.87031734


/tmp/ipykernel_60841/2894921470.py:2: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = qd_client.search(
